# Process Data
- get_batch

In [4]:
from AHDE_process_data import *
import numpy as np
from nlp_vocab import *

In [5]:
from params import *
params = Params()
params.IS_DEBUG = True
params.DATA_DIR = '../data/target_aaai-19_whole/'

In [6]:
batch_gen = ProcessData(False, params, data_path=params.DATA_DIR)

IS_TEST = False
load data : DEBUG mode


 64%|██████▎   | 127/200 [00:00<00:00, 1263.11it/s]

add pad index as : 0
[completed] load data
voca size (include _PAD_, _UNK_): 253067
create_datat_set (delimiter <EOP>):  7


  0%|          | 0/200 [00:00<?, ?it/s]

[completed] create TRAIN: 200
create_datat_set (delimiter <EOP>):  7
[completed] create VALID: 200
create_datat_set (delimiter <EOP>):  7


100%|██████████| 200/200 [00:00<00:00, 1082.75it/s]

[completed] create TEST: 200


In [4]:
en, res, enN, conN, resN,label = batch_gen.get_batch( batch_gen.test_set, 
                                                        batch_size=2, 
                                                        encoder_size=80,
                                                        context_size=15, 
                                                        encoderR_size = 160, 
                                                        is_test=True,
                                                        start_index=0)

In [5]:
voca = Vocab(batch_gen.voca)

In [6]:
batch_gen.train_set[0][2]

1.0

In [10]:
# print(voca.index2sent(batch_gen.train_set[0][0]))
# print(voca.index2sent(batch_gen.train_set[0][1][0]))
# print(voca.index2sent(batch_gen.train_set[0][1][1]))
# print(voca.index2sent(batch_gen.train_set[0][1][2]))

# AHDE model

In [1]:
import tensorflow as tf
from AHDE_Model import *
from AHDE_process_data import *
# from train_AHDE import *
from params import *
params = Params()
params.IS_DEBUG = True
params.DATA_DIR = '../data/target_aaai-19_whole/'
batch_gen = ProcessData(False, params, data_path=params.DATA_DIR)

IS_TEST = False
load data : DEBUG mode


 65%|██████▌   | 130/200 [00:00<00:00, 1298.10it/s]

add pad index as : 0
[completed] load data
voca size (include _PAD_, _UNK_): 253067
create_datat_set (delimiter <EOP>):  7


 66%|██████▌   | 131/200 [00:00<00:00, 1307.65it/s]

[completed] create TRAIN: 200
create_datat_set (delimiter <EOP>):  7


 53%|█████▎    | 106/200 [00:00<00:00, 1059.67it/s]

[completed] create VALID: 200
create_datat_set (delimiter <EOP>):  7


100%|██████████| 200/200 [00:00<00:00, 889.68it/s] 

[completed] create TEST: 200


In [2]:
batch_size = 2
encoder_size, context_size, encoderR_size = 2, 10, 3
embed_size = 10

num_layer = 1
hidden_dim = 3
num_layer_con = 1
hidden_dim_con = 5

use_glove = 0
lr = 1e-3
dr = 1.0

model_type = 0

model = AttnHrDualEncoderModel(
            params=params,
            voca_size=len(batch_gen.voca),
            batch_size=batch_size,
            encoder_size=encoder_size,
            encoderR_size = encoderR_size,
            context_size = context_size,
            num_layer=num_layer,
            lr = lr,
            hidden_dim=hidden_dim,
            num_layer_con=num_layer_con,
            hidden_dim_con=hidden_dim_con,
            use_glove=use_glove,
            embed_size = embed_size,
            fix_embed = 0
            )
model.build_graph()

[launch] create placeholders

[launch] create embedding layer
[INFO] embedding train : TRUE

[launch] create encoding layer textBi/chunkBi, textResi/chunkResi False True False False

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
[layers] bidir case
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equiva

[layers] bidir case
[launch] create attention layer
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
dim is deprecated, use axis instead
[launch] create output projection layer

Instructions for updating:
keep_dims is deprecated, use keepdims instead
[launch] create optimizer

[launch] create summary




In [4]:
saver = tf.train.Saver()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

if use_glove == 1 :
    sess.run(model.embedding_init, feed_dict={ model.embedding_placeholder: batch_gen.get_glove() })

In [5]:
raw_encoder_inputs, raw_encoderR_inputs, raw_encoder_seq, raw_context_seq, raw_encoderR_seq, raw_target_label = batch_gen.get_batch(
                                        data=batch_gen.train_set,
                                        batch_size=model.batch_size,
                                        encoder_size=model.encoder_size,
                                        context_size=model.context_size,
                                        encoderR_size=model.encoderR_size,
                                        is_test=False
                                        )

# prepare data which will be push from pc to placeholder
input_feed = {}

input_feed[model.encoder_inputs] = raw_encoder_inputs
input_feed[model.encoderR_inputs] = raw_encoderR_inputs

input_feed[model.encoder_seq_length] = raw_encoder_seq
input_feed[model.context_seq_length] = raw_context_seq
input_feed[model.encoderR_seq_length] = raw_encoderR_seq

input_feed[model.y_label] = raw_target_label
input_feed[model.dr_text_in_ph] = 1.0
input_feed[model.dr_text_out_ph] = 1.0
input_feed[model.dr_con_in_ph] = 1.0
input_feed[model.dr_con_out_ph] = 1.0

In [6]:
r1, r2 = sess.run( [
                model.batch_prob,
                model.loss
                    ],input_feed)

In [8]:
train_model(model, batch_gen, num_train_steps=5, valid_freq=3)

100%|██████████| 101/101 [00:01<00:00, 94.94it/s] 

0 mins step/seen/epoch: 3/ 6/0.03	val: 0.525  test: 0.525  troc: 0.590  loss: 70.00
